In [9]:
import numpy as np
import pandas as pd
import scipy.stats as st
pd.set_option('display.max_columns', None)
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
import missingno as msno
from sklearn.preprocessing import StandardScaler
from scipy import stats
from pandas_profiling import ProfileReport

In [10]:
#Необходимо прочитать файл, чтобы узнать какие есть первоначальные данные
realt=pd.read_csv('C:/Data Science/2_Task/Descriptive_stat/clients.csv', delimiter=',')
realt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       195 non-null    int64  
 1   Building                 195 non-null    int64  
 2   Year of sale             195 non-null    int64  
 3   Month of sale            195 non-null    int64  
 4   Type of property         195 non-null    object 
 5   Property number          195 non-null    int64  
 6   Area (ft.)               195 non-null    float64
 7   Price                    195 non-null    object 
 8   Status                   195 non-null    object 
 9   Customer ID              195 non-null    object 
 10  Entity                   195 non-null    object 
 11  Name                     195 non-null    object 
 12  Surname                  195 non-null    object 
 13  Age at time of purchase  178 non-null    float64
 14  Age Interval             1

In [11]:
#Скопируем первоначальный датафрейм, чтобы не портить оригинал
realt_copy=realt.copy()

In [12]:
realt_copy.describe()

,ID,Building,Year of sale,Month of sale,Property number,Area (ft.),Age at time of purchase,Y,M,D,Deal satisfaction
count,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,178.000000,178.000000,178.000000,178.000000,195.000000
mean,2591.425641,2.564103,2006.333333,7.271795,27.328205,899.867128,46.151685,1960.325843,6.752809,15.449438,3.605128
std,1244.617996,1.243434,1.072941,3.378674,14.830832,261.489550,12.841692,12.835575,3.450312,8.457285,1.363355
min,1002.000000,1.000000,2004.000000,1.000000,1.000000,410.710000,19.000000,1931.000000,1.000000,1.000000,1.000000
25%,2003.500000,2.000000,2006.000000,4.000000,15.000000,739.860000,37.000000,1950.250000,4.000000,8.000000,3.000000
50%,2054.000000,2.000000,2007.000000,8.000000,27.000000,785.480000,45.000000,1962.000000,6.000000,15.000000,4.000000
75%,3048.500000,3.000000,2007.000000,10.000000,39.000000,1075.460000,55.000000,1969.750000,9.750000,23.750000,5.000000
max,5051.000000,5.000000,2010.000000,12.000000,59.000000,1942.500000,76.000000,1986.000000,15.000000,31.000000,5.000000


In [13]:
realt_copy['Area'] = (realt_copy['Area (ft.)']*0.092903).round(decimals = 2)
realt_copy['Price'] = realt_copy['Price'].replace('[\$,]', '', regex = True).astype(float)
#Смотрим пропущенные значения
realt_copy.isnull().sum()

ID                          0
Building                    0
Year of sale                0
Month of sale               0
Type of property            0
Property number             0
Area (ft.)                  0
Price                       0
Status                      0
Customer ID                 0
Entity                      0
Name                        0
Surname                     0
Age at time of purchase    17
Age Interval               17
Y                          17
M                          17
D                          17
Gender                     17
Country                     0
State                      14
Purpose                     0
Deal satisfaction           0
Mortgage                    0
Source                      0
Area                        0
dtype: int64

In [14]:
#У нас маркетинговый анализ и есть такая колонка как год продажи. Моя гипотеза такова: скорее всего каждый год вносились какие-то изменения в маркетинговую стратегию, также есть колонка ипотека (возможно тоже были какие-то изменения в правилах кредитования.)
# Поэтому я посмотрю за какие года есть статистика и принимаю решение разбить портрет целевого клиента по годам. Это позволит нам отследить как менялся портрет целевого клиента. И каков он последний год, когда есть данные.
realt_copy['Year of sale'].unique()
#И так вся выборка для клиентов разных годов. Я бы для анализа данных выбрала бы следующую статегию.
#Клиентов поделить по двум категориям:
#постоянные клиенты, т.е. совершили больше одной покупки в компании
#покупки по годам (тут я не буду учитывать у нас клиент первичный или вторичный, просто буду смотреть что он купил и какой портер клиента будет в каждом коду)
#Я считаю, что важно для любых маркетинговых стратегий разделять эти две группы и рассматривать дополнительно их отдельно
#и в конце посмотерь статистику, если бы мы не разделяли по годам, а взяли бы всех клиентов одним датасетом

array([2005, 2007, 2004, 2008, 2006, 2010], dtype=int64)

In [15]:
#Выше получается, что у нас 17 нулевых строк. Я бы выкинула их, т.к. это меньге 10% от всей выборки, а отсутствующие данные яаляются ключевыми, поэтому я удалю из датасета все строки с нулевами значениями
#realt_clean = realt_copy.loc[realt_copy['Age at time of purchase'].isnull()]
#realt_clean = realt_copy.drop(np.where(realt_copy['Age at time of purchase'] == 0) [0])
a=realt_copy.fillna(0)
realt_clean = a.drop(a[a['Age at time of purchase'] == 0].index)
realt_clean.isnull().sum()
#realt_clean.info()
#Получили новый дата сет без нулевых значений
realt_c=realt_clean.copy()
realt_clean['Customer ID'].count()


178

In [18]:
#Создам новые даты сеты для каждого года.
#2004
realt_2004 = realt_c.drop(realt_c[realt_c['Year of sale'] != 2004].index)
realt_2004.info
#удаление дубликатов
realt_2004.drop_duplicates(subset=['Customer ID'])
report_2004 = ProfileReport(realt_2004, title = "Clients 2004", explorative=True, minimal=True)
report_2004.to_file("./report_2004.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
#Создам новые даты сеты для каждого года.
#2005
realt_2005 = realt_clean.drop(realt_clean[realt_clean['Year of sale'] != 2005].index)
realt_2005.info
#удаление дубликатов
realt_2005.drop_duplicates(subset=['Customer ID'])
report_2005 = ProfileReport(realt_2004, title = "Clients 2005", explorative=True, minimal=True)
report_2005.to_file("./report_2005.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
#Создам новые даты сеты для каждого года.
#2006
realt_2006 = realt_c.drop(realt_clean[realt_clean['Year of sale'] != 2006].index)
realt_2006.info
#удаление дубликатов
realt_2006.drop_duplicates(subset=['Customer ID'])
report_2006 = ProfileReport(realt_2006, title = "Clients 2006", explorative=True, minimal=True)
report_2006.to_file("./report_2006.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
#Создам новые даты сеты для каждого года.
#2007
realt_2007 = realt_c.drop(realt_clean[realt_clean['Year of sale'] != 2007].index)
realt_2007.info
#удаление дубликатов
realt_2007.drop_duplicates(subset=['Customer ID'])
report_2007 = ProfileReport(realt_2007, title = "Clients 2007", explorative=True, minimal=True)
report_2007.to_file("./report_2007.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#Создам новые даты сеты для каждого года.
#2008
realt_2008 = realt_c.drop(realt_clean[realt_clean['Year of sale'] != 2008].index)
realt_2008.info
#удаление дубликатов
realt_2008.drop_duplicates(subset=['Customer ID'])
report_2008 = ProfileReport(realt_2004, title = "Clients 2008", explorative=True, minimal=True)
report_2008.to_file("./report_2008.html")

In [ ]:
#Создам новые даты сеты для каждого года.
#2010
realt_2010 = realt_c.drop(realt_clean[realt_clean['Year of sale'] != 2010].index)
realt_2010.info
#удаление дубликатов
realt_2010.drop_duplicates(subset=['Customer ID'])
report_2010 = ProfileReport(realt_2004, title = "Clients 2010", explorative=True, minimal=True)
report_2010.to_file("./report_2010.html")

In [129]:
#Создам новые даты сеты для постоянных клиентов.
#Посмотрим, если у нас постоянные клиенты
a=realt_copy['Customer ID'].value_counts()
a.info
#Но что делать дальше с этими клиентами, у меня не появилось идей, так как я посмотрела, что их всего 20 человек.  

a.value_counts()

1    142
2     17
9      1
7      1
3      1
Name: Customer ID, dtype: int64